In [ ]:
import json
from elasticsearch import Elasticsearch, helpers
import datetime
from glob import glob
es = Elasticsearch(['http://34.95.230.124:9200/'])

def parse_date(tweet):
    tweet['created_at'] = datetime.datetime.strptime(tweet['created_at'],'%a %b %d %H:%M:%S +0000 %Y')
    tweet['user']['created_at'] = datetime.datetime.strptime(tweet['user']['created_at'],'%a %b %d %H:%M:%S +0000 %Y')
    if ('retweeted_status' in tweet):
        tweet['retweeted_status']['created_at'] = datetime.datetime.strptime(tweet['retweeted_status']['created_at'],'%a %b %d %H:%M:%S +0000 %Y')
        tweet['retweeted_status']['user']['created_at'] = datetime.datetime.strptime(tweet['retweeted_status']['user']['created_at'],'%a %b %d %H:%M:%S +0000 %Y')
    return tweet

In [ ]:
forest500settings = {
    "settings":{
        "index":{
            "mapping":{
                "total_fields":{
                    "limit":"2000"
                }
            }
        }
    }
}
es.indices.create('forest500',body=forest500settings)

In [ ]:
companies_path = glob('/Users/luiz/OneDrive/forest500/output-company/*.json')
#companies_path = glob('../data/company/*.json')
for company_path in companies_path:
    with open(company_path) as f:
        tweets = json.load(f)
    
    print(company_path)

    for tweet in tweets:
        tweet = parse_date(tweet)
        tweet['user']['type'] = 'company'
        res = es.index(index="forest500", id=tweet['id'], body=tweet)

es.indices.refresh(index="forest500")

In [ ]:
companies_path = glob('/Users/luiz/OneDrive/forest500/output/*.json')
#companies_path = glob('../data/if/*.json')
for company_path in companies_path:
    with open(company_path) as f:
        tweets = json.load(f)
    
    print(company_path)

    for tweet in tweets:
        tweet = parse_date(tweet)
        tweet['user']['type'] = 'financial'
        res = es.index(index="forest500", id=tweet['id'], body=tweet)

es.indices.refresh(index="forest500")

In [ ]:
query = {
  "query": {
    "query_string": {
      "query": "SantanderAM_UK",
      "default_field": "user.screen_name"
    }
  }
}

res = es.search(index="forest500", body=query)


In [2]:
import pandas as pd

companies = pd.read_excel('../input/forest500.xlsx',sheet_name='company')
financial = pd.read_excel('../input/forest500.xlsx',sheet_name='financial')

for index,row in companies.iterrows():
    if(pd.isna(row['Handles'])):
        continue
    handles = row['Handles'].split(',')
    for handle in handles:
        print(handle)
        query = {
            "query": {
                "term": {
                    "user.screen_name.keyword": {
                        "value": handle,
                        "boost": 1.0
                    }
                }
            },
            "_source": False
        }


        res = helpers.scan(
                        client = es,
                        query = query, 
                        index = 'forest500')

        for tweet in res:
            update_query = {
                'doc':{
                    
                        'official_name': row['Name'],
                        'type': row['Type'],
                        'subsidary': row['Subsidary'],
                        'brands': str(row['Brands']).split(','),
                        'sectors': str(row['Company sectors']).split(','),
                        'headquarters': row['Headquarters'],
                        'region': row['HQ region'],
                        'commodities_powerbroker': str(row['Commodities (powerbroker_']).split(','),
                        'segments_powerbroker': str(row['Segments (powerbroker)']).split(','),
                        'commodities_other': str(row['Commodities (Other)']).split(','),
                        'segments_other': str(row['Segments (Other)']).split(','),
                        'signatories': str(row['Sign']).split(','),
                }
            }
            try:
                es.update(index='forest500', id=tweet['_id'], body=update_query)
            except:
                print(update_query)


3M


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/elasticsearch/connection/base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.13/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


agdagro
adidas


In [ ]:
import pandas as pd

financial = pd.read_excel('../input/forest500.xlsx',sheet_name='financial')

for index,row in financial.iterrows():
    if(pd.isna(row['Handles'])):
        continue
    handles = row['Handles'].split(',')
    for handle in handles:
        print(handle)
        query = {
            "query": {
                "term": {
                    "user.screen_name.keyword": {
                        "value": handle,
                        "boost": 1.0
                    }
                }
            },
            "_source": False
        }

        res = helpers.scan(
                        client = es,
                        query = query, 
                        index = 'forest500')

        for tweet in res:
            update_query = {
                'doc':{
                        'official_name': row['Name'],
                        'type': row['Type'],
                        'subsidary': str(row['FI subsidary']),
                        'FI type': str(row['FI type']).split(','),
                        'headquarters': str(row['FI Headquarters']),
                        'region': str(row['HQ Region']),
                        'commodities': str(row['Commodities assessed for']).split(','),
                }
            }
            try:
                es.update(index='forest500', id=tweet['_id'], body=update_query)
            except:
                print(update_query)